# импорты

In [1]:
import csv
import re

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import networkx as nx

In [4]:
from tqdm import tqdm

In [5]:
import io
import os
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [7]:
from tensorboard.plugins import projector

# константы

In [8]:
data_folder = '.'

# загрузка данных

In [9]:
structures_data_embedded = pd.read_csv(f'{data_folder}/structures_data_embedded.csv')
structures_data_embedded = structures_data_embedded.dropna()
structures_data_embedded

,id,name,mrdef,embeddings
0,5392,capmatinib,Capmatinib is a kinase inhibitor indicated for...,[-0.3585707 0.6105027 -0.03888055 0.527587...
1,5393,selpercatinib,Selpercatinib is a kinase inhibitor. Selpercat...,[-0.351399 0.5350093 -0.06064523 0.446621...
2,5394,ripretinib,Ripretinib is a tyrosine kinase inhibitor that...,[-0.37962225 0.6491094 -0.04126282 0.557478...
3,5377,molnupiravir,EIDD-1931 has broad spectrum antiviral activit...,[-0.37407592 0.60611284 -0.04967265 0.512396...
4,5395,fluoroestradiol F 18,Fluoroestradiol F 18 binds ER and it is a radi...,[-0.35038048 0.5291053 -0.06157967 0.444059...
...,...,...,...,...
3183,5224,dupilumab,Dupilumab is a human monoclonal IgG4 antibody ...,[-0.3977162 0.591896 -0.06846568 0.507152...
3184,5109,urokinase,A proteolytic enzyme that converts PLASMINOGEN...,[-0.39410973 0.6405348 -0.0437251 0.549032...
3185,5433,ansuvimab,EBANGA (ansuvimab-zykl) is a Zaire ebolavirus ...,[-0.40826026 0.6270546 -0.07548815 0.528967...
3186,5434,margetuximab,"Margetuximab-cmkb, a HER2/neu receptor antagon...",[-0.39354396 0.5811391 -0.08454745 0.486416...


# готовим данные

In [10]:
# Функция для преобразования строки с разделителями пробелами и символами новой строки в список вещественных чисел
def str_to_float_list(s):
    # Заменяем символы новой строки на пробелы и удаляем скобки на концах
    clean_str = re.sub(r'[\n\s\t]+', ' ', s[1:-1])
    return np.fromstring(clean_str, sep=' ')

In [11]:
intermediate_output = np.array([str_to_float_list(s) for s in structures_data_embedded.embeddings])

# рисуем интерактивную штуку

In [12]:
log_dir = './logs/mytensor/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

with open(os.path.join(log_dir, 'metadata.tsv'), 'w') as f:
    for idx, row in structures_data_embedded.iterrows():
        f.write(f"{row['name']}\n")

In [13]:
np.savetxt("./logs/mytensor/data.tsv", intermediate_output,  
              delimiter = "\t")

In [14]:
feature_vectors = tf.Variable(intermediate_output)
checkpoint = tf.train.Checkpoint(embedding=feature_vectors)
checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
embedding.tensor_path = 'data.tsv'
projector.visualize_embeddings(log_dir, config)

In [15]:
%load_ext tensorboard

In [16]:
%tensorboard --logdir ./logs/mytensor